In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
import time
import numpy as np
import pylab as pl
import tqdm

pd.set_option('display.max_columns', None)

<h1>Q3: Low Birth Weight Causes Infant Mortality?</h1>

In [3]:
twins = pd.read_csv('twins.gz')
print(twins.shape)

(59052, 35)


In [4]:
# Let us look at the first few rows.
twins.head(n=10)

,mort,csex,dbirwt,dmage,mrace,dmeduc,dmar,dlivord,mpcb,anemia,cardiac,lung,diabetes,herpes,hydra,hemo,chyper,phyper,eclamp,incervix,pre4000,preterm,renal,rh,uterine,othermr,tobacco,alcohol,dfage,frace,dfeduc,infant_id,pair_id,term,id
0,0,1,2601.0,22,black,college,1,2.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,30.0,black,highschool,2,3,0.0,1.0
1,0,2,3069.0,22,black,college,1,2.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,30.0,black,highschool,3,3,0.0,2.0
2,0,1,2948.0,24,white,highschool,1,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.0,white,college,4,5,0.0,1.0
3,0,2,2948.0,24,white,highschool,1,3.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30.0,white,college,5,5,0.0,2.0
4,0,1,3345.0,32,white,highschool,1,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.0,white,highschool,8,9,0.0,1.0
5,0,2,2863.0,32,white,highschool,1,3.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.0,white,highschool,9,9,0.0,2.0
6,0,2,2098.0,31,white,morethancollege,1,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.0,white,highschool,10,11,0.0,1.0
7,0,2,1985.0,31,white,morethancollege,1,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.0,white,highschool,11,11,0.0,2.0
8,0,1,2126.0,24,white,highschool,1,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,22.0,white,highschool,12,13,0.0,1.0
9,0,1,1985.0,24,white,highschool,1,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,22.0,white,highschool,13,13,0.0,2.0


<h3>a) Why is this data well-suited for matching?</h3>

Note: No coding is required to answer 3a. Cf. report.

<h3>b) Matching for ATE</h3>

In [5]:
# Threshold under which birth weight is considered to be low.
threshold = 2700

# Filter for babies whose birth weight is below the threshold.
twins_dbirwt = twins[twins['dbirwt'] < threshold]

In [6]:
# Get the counts per twin pair id.
twins_dbirwt_pairs_counts = twins_dbirwt['pair_id'].value_counts()
# Obtain the list of eligible pairs (i.e. there is only one twin whose birth weight was below the threshold).
twins_dbirwt_eligible_pairs = list(twins_dbirwt_pairs_counts[twins_dbirwt_pairs_counts == 1].index)

In [7]:
# Select eligible pairs.
twins_eligible_pairs = twins.loc[twins['pair_id'].isin(twins_dbirwt_eligible_pairs)]
twins_eligible_pairs.shape

(19800, 35)

In [8]:
# Look at the first few rows and check that there are only pairs with one baby below and above the birth weight threshold.
twins_eligible_pairs.head(n=10)

,mort,csex,dbirwt,dmage,mrace,dmeduc,dmar,dlivord,mpcb,anemia,cardiac,lung,diabetes,herpes,hydra,hemo,chyper,phyper,eclamp,incervix,pre4000,preterm,renal,rh,uterine,othermr,tobacco,alcohol,dfage,frace,dfeduc,infant_id,pair_id,term,id
0,0,1,2601.0,22,black,college,1,2.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,30.0,black,highschool,2,3,0.0,1.0
1,0,2,3069.0,22,black,college,1,2.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,30.0,black,highschool,3,3,0.0,2.0
12,0,1,3044.0,22,white,highschool,1,2.0,2.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,24.0,white,highschool,16,17,0.0,1.0
13,0,1,2361.0,22,white,highschool,1,3.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,24.0,white,highschool,17,17,0.0,2.0
20,0,2,2594.0,20,white,highschool,1,2.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,24.0,white,college,30,31,1.0,1.0
21,0,1,2707.0,20,white,highschool,1,3.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,24.0,white,college,31,31,1.0,2.0
22,0,1,2410.0,26,white,college,1,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.0,white,college,32,33,0.0,1.0
23,0,1,2835.0,26,white,college,1,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29.0,white,college,33,33,0.0,2.0
58,0,1,2600.0,32,white,highschool,1,4.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,29.0,white,highschool,78,79,1.0,1.0
59,0,2,3550.0,32,white,highschool,1,5.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,29.0,white,highschool,79,79,1.0,2.0


In [9]:
# Create a new column to characterize treatment.
twins_eligible_pairs['treatment'] = 1*(twins_eligible_pairs['dbirwt'] < 2700)

C:\Users\Marie CHARPIGNON\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [73]:
# Check that there are twins_eligible_pairs.shape[0]/2 treatment babies.
n_treatment_cases = twins_eligible_pairs['treatment'].sum()
print("Number of treatment babies: ", n_treatment_cases)
print("Does this number match with half the number of eligible pairs? ", n_treatment_cases == twins_eligible_pairs.shape[0]/2)

Number of treatment babies:  9900
Does this number match with half the number of eligible pairs?  True


In [12]:
# Mortality among twin babies that are below threshold.
def te1_func(x):
    return (x[x['treatment'] == 1]['mort'])
# Mortality among twin babies that are above threshold.
def te0_func(x):
    return (x[x['treatment'] == 0]['mort'])

# Mortality tables, by pair id (filtered for eligible ones) and treatment category (0 or 1).
df1 = pd.DataFrame(twins_eligible_pairs.groupby(['pair_id','treatment']).apply(te1_func))
df0 = pd.DataFrame(twins_eligible_pairs.groupby(['pair_id','treatment']).apply(te0_func))

In [15]:
# Check that df1 table is well ordered, before proceeding to comparison and final estimation.
df1.head()

,,,mort
pair_id,treatment,,
3,1,0,0
17,1,13,0
31,1,20,0
33,1,22,0
79,1,58,0


In [16]:
# Check that df0 is well ordered, before proceeding to comparison and final estimation.
df0.tail()

,,,mort
pair_id,treatment,,
90167,0,58999,0
90187,0,59019,0
90203,0,59034,0
90207,0,59038,0
90213,0,59045,0


In [17]:
# Estimate the Average Treatment Effect (ATE).
ate = (df1.values - df0.values).mean()

print("Average Treatment Effect (ATE): ", ate)

Average Treatment Effect (ATE):  0.03272727272727273


<h3>c) Generalizability from Counterfactual Twins to Singletons</h3>

In [18]:
singletons = pd.read_csv('singletons.csv')
singletons.head(n=10)

,Unnamed: 0,Unnamed: 0.1,mort,csex,dbirwt,dmage,mrace,dmeduc,dmar,dlivord,mpcb,anemia,cardiac,lung,diabetes,herpes,hydra,hemo,chyper,phyper,eclamp,incervix,pre4000,preterm,renal,rh,uterine,othermr,tobacco,alcohol,dfage,frace,dfeduc,infant_id,term,m_race_black,m_race_other,m_race_white,m_edu_college,m_edu_elementary,m_edu_highschool,m_edu_morethancollege,m_edu_noedu
0,0,0,0,1,2977.0,29,white,highschool,0,2.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,22.0,white,highschool,0,0.0,0,0,1,0,0,1,0,0
1,1,1,0,2,3912.0,25,white,college,1,2.0,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.0,white,college,1,1.0,0,0,1,1,0,0,0,0
2,2,2,0,1,3317.0,36,white,morethancollege,1,3.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33.0,white,college,2,0.0,0,0,1,0,0,0,1,0
3,3,3,0,2,2963.0,30,white,college,1,2.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,31.0,white,college,3,0.0,0,0,1,1,0,0,0,0
4,4,4,0,2,3572.0,25,white,highschool,1,3.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26.0,white,highschool,4,2.0,0,0,1,0,0,1,0,0


In [21]:
singletons_mort_rate = singletons['mort'].mean()
print("Mortality rate among singletons: ", singletons_mort_rate)

Mortality rate among singletons:  0.005049114109978886


In [22]:
twins_mort_rate = twins['mort'].mean()
print("Mortality rate among twins: ", twins_mort_rate)

Mortality rate among twins:  0.034613560929350404


In [23]:
print("Mortality rate among eligible twin pairs: ", twins_eligible_pairs['mort'].mean())
print("Mortality rate among eligible twin pair babies whose birth weight is below the threshold: ", twins_eligible_pairs[twins_eligible_pairs['treatment'] == 1]['mort'].mean())
print("Mortality rate among eligible twin pair babies whose birth weight is above the threshold: ", twins_eligible_pairs[twins_eligible_pairs['treatment'] == 0]['mort'].mean())

Mortality rate among eligible twin pairs:  0.021616161616161617
Mortality rate among eligible twin pair babies whose birth weight is below the threshold:  0.03797979797979798
Mortality rate among eligible twin pair babies whose birth weight is above the threshold:  0.0052525252525252525


<h1>Q4: Smoking During Pregnancy Causes Low Birth Weight?</h1>


In [28]:
# Features
X = ['dmage', 'dmar', 'dlivord', 'anemia', 'cardiac', 'lung', 'diabetes', 'herpes',\
     'hydra', 'hemo', 'chyper', 'phyper', 'eclamp', 'incervix', 'pre4000', 'preterm', \
     'renal', 'rh', 'uterine', 'othermr', 'alcohol',\
     'm_race_black', 'm_race_other', 'm_race_white', \
     'm_edu_college', 'm_edu_elementary', 'm_edu_highschool', 'm_edu_morethancollege', 'm_edu_noedu']
# Treatment
T = 'tobacco'
# Outcome
O = 'dbirwt'

<h3>a) Naive Difference in Cohorts</h3>


In [25]:
# Average birth weight among babies of smoking vs. non-smoking mothers
cohort_smoking = singletons[singletons['tobacco'] == 1]['dbirwt'].mean()
cohort_non_smoking = singletons[singletons['tobacco'] == 0]['dbirwt'].mean()

In [26]:
print("Average birth weight among babies of smoking mothers: ", cohort_smoking)
print("Average birth weight among babies of non-smoking mothers: ", cohort_non_smoking)
print("Average difference between birth weights of babies of smoking vs. non-smoking mothers (non-adjusted): ", cohort_smoking - cohort_non_smoking)

Average birth weight among babies of smoking mothers:  3335.3483711747285
Average birth weight among babies of non-smoking mothers:  3500.1907806611775
Average difference between birth weights of babies of smoking vs. non-smoking mothers (non-adjusted):  -164.84240948644901


<h3>b) Covariate Adjustment</h3>

In [32]:
# Initiate linear regression model.
lr = LinearRegression()
# Expand the set of covariates to include treatment variable.
XT = X + [T]
# Fit regression model to the data.
lr.fit(singletons[XT], singletons[O]) 

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [33]:
singletons['zeros'] = 0
singletons['ones'] = 1
X1 = X + ['ones']
X0 = X + ['zeros']

# Outcome among treatment babies
predicted1 = lr.predict(singletons[X1])
# Outcome among control babies
predicted0 = lr.predict(singletons[X0])

In [34]:
# Difference between treatment and control babies
linear_regression_diff = (predicted1 - predicted0).sum()/singletons.shape[0]
print("Linear regression birth weight difference between babies born from smoking vs. non-smoking mothers: ", linear_regression_diff)

-188.40670155765795

In [67]:
# Feature importance - Results from linear regression
for variable,coef in sorted(zip(XT,lr.coef_), key=lambda x: x[1]):
    if coef != 0:
        print(variable,coef)

incervix -526.1005831471906
uterine -473.851485883662
eclamp -447.6475424712485
preterm -421.46221141069435
hydra -244.64018151033451
renal -231.7674063959836
tobacco -188.406701557658
rh -182.92580175052657
chyper -167.81773848669556
lung -156.47851591390443
phyper -138.75000069921452
m_edu_noedu -131.55344085997604
m_race_other -94.62546500141941
m_edu_elementary -64.06983307912597
othermr -56.94849751031945
cardiac -30.67194220343216
m_race_black -29.583467817332043
anemia -13.156000468474785
dmage -1.1548056006939813
herpes 6.9326027081633415
m_edu_highschool 26.667402242136124
dlivord 43.709821569074336
dmar 69.90597853894708
alcohol 73.07964775058294
m_edu_college 81.90820082908382
m_edu_morethancollege 87.0476708677796
hemo 95.03911663819046
diabetes 102.57040146458263
m_race_white 124.20893281875703
pre4000 523.1792889261832


<h3>c) Propensity Score Re-Weighting</h3>

In [68]:
# Number of iterations
n_iter = 1000

# Logistic regression model
logr = LogisticRegression(solver='lbfgs',max_iter=n_iter)
logr.fit(singletons[X],singletons[T])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [69]:
# Propensity score weights - Results from logistic regression
propensity_score = logr.predict_proba(singletons[X])
singletons['propensity_score'] = propensity_score[:,1]

In [70]:
# Sum among treatment babies
sum_among_treated = sum(singletons[singletons[T] == 1][O]/singletons[singletons[T] == 1]['propensity_score'])
# Sum among control babies
sum_among_control = sum(singletons[singletons[T] == 0][O]/(1-singletons[singletons[T] == 0]['propensity_score']))

# Difference between treatment and control babies
prop_score_weighted_diff = (sum_among_treated - sum_among_control)/singletons.shape[0]

In [71]:
# Propensity score weighting results
print("Propensity score weighted birth weight difference between babies born from smoking vs. non-smoking mothers: ", prop_score_weighted_diff)

Propensity score weighted birth weight difference between babies born from smoking vs. non-smoking mothers:  -407.1220602317252
